<a href="https://colab.research.google.com/github/danhorsley/DS-Unit-2-Sprint-2-Regression/blob/master/Copy_of_LS_DS_411A_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science — Tree Ensembles_ 

# Decision Trees Assignment

## Part 1: House Price Regression

Apply decision trees to the Ames housing dataset you've worked with this week!

- Try multiple features
- Try features you've engineered
- Try different `max_depth` paramaters
- What's the best Test Root Mean Squared Error you can get? *Share with your cohort on Slack!*
- What's a cool visualization you can make? *Share with your cohort on Slack!*

In [1]:
#copying my code from last night for clean data set

import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv')
#dropping ids and mapping quality to an order of numbers
df=df.drop(['Id','MasVnrArea'],axis=1)
qual_dict={'NA':-1,'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4}
cat_quals=['ExterQual','ExterCond','KitchenQual','BsmtQual','BsmtCond','GarageQual','GarageCond','HeatingQC','FireplaceQu']#,'Fence']
for col in cat_quals:
  df[col]=df[col].map(qual_dict)
#new build category and renovated category using YrSold, 'YearBuilt', 'YearRemodAdd
df['new_build']=np.where((df['YrSold']==df['YearBuilt']),1,0)
df['reno']=np.where((df['YearBuilt']!=df['YearRemodAdd']),1,0)
df['SF_per_room']=(df['1stFlrSF']+df['2ndFlrSF'])/df['TotRmsAbvGrd']
df['bth_per_SF']=(df['BsmtFullBath']+0.5*df['BsmtHalfBath']+ df['FullBath']+0.5*df['HalfBath'])/df['GrLivArea']
df['rel_lot_size']=df['LotArea']/df['GrLivArea'] 
df['fireplace_per_SF']=df['Fireplaces']/df['GrLivArea']
#make a dictionary of urban density and use it to make new column
density_dict=df.groupby('Neighborhood', as_index=False)['LotArea'].mean().set_index('Neighborhood').to_dict()['LotArea']
df['urban_density']=df['Neighborhood'].map(density_dict)
#same for area age
age_dict=df.groupby('Neighborhood', as_index=False)['YearBuilt'].mean().set_index('Neighborhood').to_dict()['YearBuilt']
df['area_age']=df['Neighborhood'].map(age_dict)
#now an area quality dictionary
area_qual_dict=df.groupby('Neighborhood', as_index=False)['OverallQual'].mean().set_index('Neighborhood').to_dict()['OverallQual']
df['area_qual']=df['Neighborhood'].map(area_qual_dict)
#noise polution list and check
noise_high_list=['Artery','RRAn','RRAe']
df['noisy']=np.where(df['Condition1'].isin(noise_high_list),1,0)
df['ln_price'] = np.log(df['SalePrice'])
df_coded=df.copy()
for col in df_coded.columns:
  if df_coded[col].dtype=='O':
    df_coded[col] = pd.Categorical(df_coded[col])
    df_coded[col] = df_coded[col].cat.codes
    
df_coded['LotFrontage']=df_coded['LotFrontage'].fillna(df_coded['LotFrontage'].mean())
df_coded['FireplaceQu']=df_coded['FireplaceQu'].fillna(0)
df_coded['GarageQual']=df_coded['GarageQual'].fillna(0)
df_coded['GarageCond']=df_coded['GarageCond'].fillna(0)
df_coded['BsmtQual']=df_coded['BsmtQual'].fillna(0)
df_coded['BsmtCond']=df_coded['BsmtCond'].fillna(0)
df_coded['GarageYrBlt']=np.where(df_coded['GarageYrBlt'].isna()==True,df_coded['YearBuilt'],df_coded['GarageYrBlt'])
corr_list=[]
for col in df_coded.columns:
  aaa=(col,df_coded['ln_price'].corr(df_coded[col]))
  corr_list.append(aaa)
  
corr_list.sort(key=lambda x: x[1])
lin_feats=[corr_list[-12:-2][i][0] for i in range(0,len(corr_list[-12:-2]))]
additional_features=[corr_list[-22:-12][i][0] for i in range(0,len(corr_list[-22:-12]))]
even_more_feats=[corr_list[-32:-22][i][0] for i in range(0,len(corr_list[-22:-12]))]
poly_feats=[]
for ft in (lin_feats+additional_features+even_more_feats):
  name = f'{ft} **2'
  df_coded[name]=df_coded[ft]**2
  poly_feats.append(name)
ft_list=['SalePrice','ln_price','FullBath', 'YearBuilt', 'Functional', 'ScreenPorch', 'HeatingQC **2', 'SaleCondition', 'OverallCond', 'BldgType', 'GrLivArea **2', 'BsmtCond', '2ndFlrSF', 'BsmtFinSF2', 'CentralAir', 'WoodDeckSF', 'OverallQual', 'FullBath **2', 'BsmtFinSF1', 'LowQualFinSF', '2ndFlrSF **2', 'Fireplaces', 'GarageQual', 'KitchenQual **2', '1stFlrSF', 'bth_per_SF', 'area_qual **2', 'GarageCars **2', 'KitchenQual', 'rel_lot_size']

dff=df_coded[ft_list].copy()
dff.head()

,SalePrice,ln_price,FullBath,YearBuilt,Functional,ScreenPorch,HeatingQC **2,SaleCondition,OverallCond,BldgType,...,2ndFlrSF **2,Fireplaces,GarageQual,KitchenQual **2,1stFlrSF,bth_per_SF,area_qual **2,GarageCars **2,KitchenQual,rel_lot_size
0,208500,12.247694,2,2003,6,0,16,4,5,0,...,729316,0,2.0,9,856,0.002047,44.089600,4,3,4.941520
1,181500,12.109011,2,1976,6,0,16,4,8,0,...,0,1,2.0,4,1262,0.001981,45.256198,4,2,7.606973
2,223500,12.317167,2,2001,6,0,16,4,5,0,...,749956,1,2.0,9,920,0.001960,44.089600,4,3,6.298992
3,140000,11.849398,1,1915,6,0,9,0,5,0,...,571536,1,2.0,9,961,0.001165,39.369473,9,3,5.562027
4,250000,12.429216,2,2000,6,0,16,4,5,0,...,1108809,1,2.0,9,1145,0.001592,62.834622,9,3,6.487716


In [0]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz

In [17]:
X = dff.copy().drop(['SalePrice','ln_price'],axis=1)
y = dff['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=42)
depth_range=[5,10,15,20,25,30,100]
for depth in depth_range:
  tree = DecisionTreeRegressor(max_depth=depth)
  tree.fit(X_train, y_train)
  y_pred=tree.predict(X_test)
  print('Accuracy at max depth ',depth,'was ', tree.score(X_test, y_test))

Accuracy at max depth  5 was  0.7714936913646108
Accuracy at max depth  10 was  0.7285219171863486
Accuracy at max depth  15 was  0.735503947617907
Accuracy at max depth  20 was  0.7952084626808907
Accuracy at max depth  25 was  0.7450088796044672
Accuracy at max depth  30 was  0.7857643248193276
Accuracy at max depth  100 was  0.7911874668931284


In [26]:
#let's try on teh whole data set, not stripped down.  accuracy goes down on the test data after 15 max depth
#depth 6,7,8 are teh best after a number of re runs
X = df_coded.copy().drop(['SalePrice','ln_price'],axis=1)
y = df_coded['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=42)
depth_range=range(5,10)
for depth in depth_range:
  tree = DecisionTreeRegressor(max_depth=depth)
  tree.fit(X_train, y_train)
  y_pred=tree.predict(X_test)
  print('Accuracy at max depth ',depth,'was ', tree.score(X_test, y_test))

Accuracy at max depth  5 was  0.8122097434094453
Accuracy at max depth  6 was  0.8078509439144425
Accuracy at max depth  7 was  0.7976966727576911
Accuracy at max depth  8 was  0.8163227290873929
Accuracy at max depth  9 was  0.7590649121188948


In [63]:
X = df_coded.copy().drop(['SalePrice','ln_price'],axis=1)
y = df_coded['SalePrice']
#y = df_coded['ln_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=42)
tree = DecisionTreeRegressor(max_depth=8)
tree.fit(X_train, y_train)
y_pred=tree.predict(X_test)
print('Accuracy at max depth ',8,'was ', tree.score(X_test, y_test))
importances=pd.DataFrame(X.columns)
importances.columns=['feature']
importances['importance']=tree.feature_importances_
importances=importances.sort_values(by='importance',ascending=False)
importances[:12]
#tree.feature_importances_

Accuracy at max depth  8 was  0.793659438265949


,feature,importance
97,OverallQual **2,0.539445
86,area_qual,0.122590
95,GrLivArea **2,0.065037
88,1stFlrSF **2,0.038882
44,GrLivArea,0.030228
109,2ndFlrSF **2,0.030034
36,TotalBsmtSF,0.020041
32,BsmtFinSF1,0.013771
11,Neighborhood,0.013155
106,YearBuilt **2,0.009877


In [99]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
feats=list(importances['feature'][:5])
X = df_coded[feats]
y = df_coded['SalePrice']
#y = df_coded['ln_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=42)
depth_range=range(5,9)
for depth in depth_range:
  tree = DecisionTreeRegressor(max_depth=depth)
  tree.fit(X_train, y_train)
  y_pred=tree.predict(X_test)
  print('Accuracy at max depth ',depth,'was ', tree.score(X_test, y_test), 'and rmse of ',(np.sqrt(mean_squared_error(y_test, y_pred))))
 # print('rmse = (np.sqrt(mean_squared_error(y_test, y_pred))))

Accuracy at max depth  5 was  0.8538894395754 and rmse of  33477.091581731045
Accuracy at max depth  6 was  0.8599895360974187 and rmse of  32770.80912263834
Accuracy at max depth  7 was  0.8579818677385268 and rmse of  33004.92996176398
Accuracy at max depth  8 was  0.84053235203154 and rmse of  34973.83135161158


## Part 2 / Stretch: "Play Tennis" Classification

We'll reproduce the "Play Tennis" example from Ross Quinlan's 1986 paper, [Induction of Decison Trees](https://link.springer.com/content/pdf/10.1007%2FBF00116251.pdf).

[According to Wikipedia](https://en.wikipedia.org/wiki/Ross_Quinlan), "John Ross Quinlan is a computer science researcher in data mining and decision theory. He has contributed extensively to the development of decision tree algorithms, including inventing the canonical C4.5 and ID3 algorithms."

#### "Table 1 shows a small training set"

In [44]:
import pandas as pd

columns = 'No. Outlook Temperature Humidity Windy PlayTennis'.split()

raw = """1 sunny hot high false N
2 sunny hot high true N
3 overcast hot high false P
4 rain mild high false P
5 rain cool normal false P
6 rain cool normal true N
7 overcast cool normal true P
8 sunny mild high false N
9 sunny cool normal false P
10 rain mild normal false P
11 sunny mild normal true P
12 overcast mild high true P
13 overcast hot normal false P
14 rain mild high true N"""

data = [row.split() for row in raw.split('\n')]
tennis = pd.DataFrame(data=data, columns=columns).set_index('No.')
tennis['PlayTennis'] = (tennis['PlayTennis'] == 'P').astype(int)

tennis

,Outlook,Temperature,Humidity,Windy,PlayTennis
No.,,,,,
1,sunny,hot,high,false,0
2,sunny,hot,high,true,0
3,overcast,hot,high,false,1
4,rain,mild,high,false,1
5,rain,cool,normal,false,1
6,rain,cool,normal,true,0
7,overcast,cool,normal,true,1
8,sunny,mild,high,false,0
9,sunny,cool,normal,false,1


#### "A decision tree that correctly classifies each object in the training set is given in Figure 2."

<img src="https://i.imgur.com/RD7d0u0.png" height="300">

In this dataset, the tennis player decided to play on 64% of the days, and decided not to on 36% of the days.

In [65]:
tennis['PlayTennis'].value_counts(normalize=True) * 100

1    64.285714
0    35.714286
Name: PlayTennis, dtype: float64

The tennis player played on 100% of the overcast days, 40% of the sunny days, and 60% of the  rainy days

In [66]:
tennis.groupby('Outlook')['PlayTennis'].mean() * 100

Outlook
overcast    100.0
rain         60.0
sunny        40.0
Name: PlayTennis, dtype: float64

On sunny days, the tennis player's decision depends on the humidity. (The Outlook and Humidity features interact.)

In [67]:
sunny = tennis[tennis['Outlook']=='sunny']
sunny.groupby('Humidity')['PlayTennis'].mean() * 100

Humidity
high        0
normal    100
Name: PlayTennis, dtype: int64

On rainy days, the tennis player's decision depends on the wind. (The Outlook and Windy features interact.)

In [68]:
rainy = tennis[tennis['Outlook']=='rain']
rainy.groupby('Windy')['PlayTennis'].mean() * 100

Windy
false    100
true       0
Name: PlayTennis, dtype: int64

#### Before modeling, we will ["encode" categorical variables, using pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html?highlight=get_dummies#computing-indicator-dummy-variables)

In [75]:
y = tennis['PlayTennis']
X = pd.get_dummies(tennis.drop(columns='PlayTennis'))
y

No.
1     0
2     0
3     1
4     1
5     1
6     0
7     1
8     0
9     1
10    1
11    1
12    1
13    1
14    0
Name: PlayTennis, dtype: int64

## Train a Decision Tree Classifier
Get a score of 100% (accuracy)

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [89]:
y = tennis['PlayTennis']
X = pd.get_dummies(tennis.drop(columns='PlayTennis'))
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=52)
depth_range=[1,2,3,4,5,6,7]
for depth in depth_range:
  clf = DecisionTreeClassifier(max_depth=depth)
  clf.fit(X_train, y_train)
  y_pred=clf.predict(X_test)
  print('Accuracy at max depth ',depth,'was ', clf.score(X_test, y_test))

Accuracy at max depth  1 was  0.3333333333333333
Accuracy at max depth  2 was  1.0
Accuracy at max depth  3 was  0.6666666666666666
Accuracy at max depth  4 was  1.0
Accuracy at max depth  5 was  1.0
Accuracy at max depth  6 was  1.0
Accuracy at max depth  7 was  1.0


## Compare to Logistic Regression
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [91]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, test_size=0.30, random_state=42)
logistic = LogisticRegression(solver='lbfgs')
logistic.fit(X_train, y_train)
print('Accuracy', tree.score(X_test, y_test))

Accuracy 1.0


## Visualize the tree
https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html